Using the `jwst_dev` conda environment on `azonca` home on Galactica

In [ ]:
import numpy as np

In [ ]:
from pyhocon import ConfigFactory

conf = ConfigFactory.parse_file('empty-subscribe-model.conf')

In [ ]:
conf = dict(conf)
conf["subscribe"] = dict(conf["subscribe"])

In [ ]:
expected_conf = ConfigFactory.parse_file('IRIS-Model-Files/drs/drs-assembly/subscribe-model.conf')

In [ ]:
import pyhocon

In [ ]:
telemetry = conf['subscribe']['events']

In [ ]:
import pandas as pd

In [ ]:
master_sheet = pd.read_csv("IRIS_telemetry_and_FITS_metadata.csv")

In [ ]:
events = master_sheet[master_sheet.isEvent == 1]

In [ ]:
events["Duplicate"] = events.duplicated(subset=["subsystem", "component", "name"])

In [ ]:
assert len(expected_conf['subscribe']['events']) == len(events) - events.Duplicate.sum()

In [ ]:
from collections import OrderedDict

In [ ]:
for row in events[events.Duplicate == 0].iterrows():
    event = OrderedDict()
    for key in ["subsystem", "component", "name", "requiredRate", "usage"]:
        if isinstance(row[1][key], str) or not np.isnan(row[1][key]):
            event[key] = row[1][key]
    conf['subscribe']['events'].append(event)

In [ ]:
len(conf['subscribe']['events'])

In [ ]:
generated_config = ConfigFactory.from_dict(conf)

In [ ]:
from pyhocon.converter import HOCONConverter
with open("generated-subscribe-model.conf", "w") as f:
    f.write(HOCONConverter().to_hocon(generated_config))

In [ ]:
!cp generated-subscribe-model.conf IRIS-Model-Files/drs/drs-assembly/subscribe-model.conf